# SageMaker Model Deployment

This notebook demonstrates how to deploy a trained XGBoost model using AWS SageMaker.

## Task 1: Setup Environment

In [ ]:
# Install dependencies
%pip install seaborn
%reset -f

import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
boto3_session = boto3.Session()
sagemaker_client = boto3_session.client('sagemaker')
sagemaker_runtime = boto3_session.client('sagemaker-runtime')
s3_client = boto3.client('s3')

## Task 2: Check if S3 Bucket and Test Data File Exist

In [ ]:
# Check if S3 bucket exists
from botocore.exceptions import ClientError


def check_s3_bucket(bucket_name: str) -> bool:
    """Check if the S3 bucket exists and is accessible.
    
    Args:
        bucket_name (str): Name of the S3 bucket.
    
    Returns:
        bool: True if bucket exists and accessible, False otherwise.
    """
    s3 = boto3.client('s3')
    try:
        s3.head_bucket(Bucket=bucket_name)
        return True
    except ClientError:
        return False


def check_s3_object(bucket_name: str, key: str) -> bool:
    """Check whether an S3 object exists and is accessible."""
    try:
        s3_client.head_object(Bucket=bucket_name, Key=key)
        return True
    except ClientError as e:
        error_code = e.response["Error"]["Code"]
        if error_code in ("404", "NoSuchKey"):
            return False
        elif error_code == "403":
            raise PermissionError(f"Access denied for s3://{bucket_name}/{key}")
        else:
            raise


def get_user_input(prompt: str) -> str:
    """Prompt user for input and ensure it's not empty.
    
    Args:
        prompt (str): Prompt text to display.
    
    Returns:
        str: User input.
    """
    while True:
        value = input(prompt).strip()
        if value:
            return value
        print("Input cannot be empty. Please try again.")


# -------------------------
# Interactive inputs
# -------------------------
bucket_name = get_user_input("Enter the S3 bucket name: ")
test_data_filepath = get_user_input("Enter 'test' data file path which contains test data: ")

# -------------------------
# Check bucket existence
# -------------------------
if not check_s3_bucket(bucket_name):
    raise ValueError(f"S3 Bucket '{bucket_name}' does not exist or you don't have access!")

# -------------------------
# Check 'test' data file existence
# -------------------------
if not check_s3_object(bucket_name, test_data_filepath):
    raise ValueError(f"'test' data file '{test_data_filepath}' does not exist or you don't have access!")

print(f"S3 Bucket '{bucket_name}' exists ✅")
print(f"'test' data file '{test_data_filepath}' exists ✅")

## Task 3: Download Test Dataset from S3